In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from typing import *

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.basics import Instrument
from qubx.core.series import Quote, Bar
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.data.helpers import loader
from qubx.core.basics import Subtype
from qubx.data.readers import DataReader, DataTransformer, AsPandasFrame, AsTimestampedRecords, RestoreTicksFromOHLC
from qubx.backtester.queue import SimulatedDataQueue, DataLoader

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [2]:
s1 = lookup.find_symbol("BINANCE.UM", "BTCUSDT")
s2 = lookup.find_symbol("BINANCE.UM", "ETHUSDT")
s3 = lookup.find_symbol("BINANCE.UM", "LTCUSDT")

In [63]:
ld = loader("BINANCE.UM", "1h", source="csv::../tests/data/csv_1h", n_jobs=1)
# ld = loader("BINANCE.UM", "1h", source="mqdb::xlydian-data", n_jobs=1)

### Test DataLoader

In [91]:
q = SimulatedDataQueue()
l1 = DataLoader(RestoreTicksFromOHLC(), ld, s1, "1d")
l2 = DataLoader(RestoreTicksFromOHLC(), ld, s2, "1d")
q += l1
q += l2

In [ ]:
k = 0
itr = q.create_iterable("2023-06-01", "2023-06-02")
for t in itr:
    print(f"{pd.Timestamp(t[2].time, 'ns')}     \t {t[0].symbol}\t{t[2]}")
    if k == 10:
        print("+ " * 10)
        q += (l3:=DataLoader(RestoreTicksFromOHLC(), ld, s3, "1d"))

    if k == 30:
        print("- " * 10)
        q -= l3
    k+=1

---

### <font color='gold'>Impl. multi-symbol</font>

In [ ]:
import heapq
from collections import defaultdict
from typing import Any, Iterator, Iterable, List

In [480]:
class IndexedObjects:
    _obj_to_index: Dict[Any, int]
    _index_to_obj: Dict[int, Any]
    _last_attached_idx: int
    _removed_obj_indices: Set[int]

    def __init__(self):
        self._obj_to_index = {}
        self._index_to_obj = {}
        self._last_attached_idx = 0
        self._removed_obj_indices = set()

    def contains(self, obj: Any) -> bool:
        return obj in self._obj_to_index

    def add_value(self, obj: Any) -> int:
        if not self.contains(obj):
            self._last_attached_idx += 1
            self._index_to_obj[self._last_attached_idx] = obj
            self._obj_to_index[obj] = self._last_attached_idx
        else:
            return -1
        return self._last_attached_idx

    def remove_value(self, obj: Any) -> int:
        idx_to_remove = -1
        if self.contains(obj):
            self._index_to_obj.pop(idx_to_remove := self._obj_to_index.pop(obj))
            self._removed_obj_indices.add(idx_to_remove)
        return idx_to_remove

    def get_value_by_index(self, idx: int) -> Any:
        return self._index_to_obj.get(idx)

    def get_index_of_value(self, obj: Any) -> Any:
        return self._obj_to_index.get(obj, -1)

    def items(self) -> Iterator[Tuple[int, Any]]:
        return ((idx, obj) for idx, obj in self._index_to_obj.items() if idx not in self._removed_obj_indices)

    def values(self) -> List[Any]:
        return list(self._index_to_obj.values())

    def indices(self) -> List[Any]:
        return list(self._index_to_obj.keys())

    def is_removed(self, idx: int) -> bool:
        return idx in self._removed_obj_indices

    def __str__(self) -> str:
        _r = ""
        for i, o in self.items():
            _r += f"[{i}]: {str(o)}\n"
        if self._removed_obj_indices:
            _r += "removed: " + ','.join([f"{-i}" for i in self._removed_obj_indices])
        return _r

    def __repr__(self) -> str:
        return str(self)

In [ ]:
ivs = IndexedObjects()
ivs.add_value("Test 1")
ivs.add_value("Test 2")
ivs.add_value("Test 3")
ivs

In [ ]:
ivs.values()

In [ ]:
for _i, v in ivs.items():
    print(_i, v)

In [ ]:
ivs.remove_value("Test 2")
ivs.remove_value("Test 1")
print(ivs)

In [545]:
class SimulationDataLoader:
    def __init__(
        self,
        reader: DataReader,
        subscription: str,
        instruments: List[Instrument],
        warmup_period: str | None = None,
        chunksize: int = 5000,
    ):
        self._instruments = IndexedObjects()
        self._reader = reader
        self._subscription = subscription
        self._subtype, self._subparams = Subtype.from_str(subscription)
        self._warmup_period = warmup_period
        self._warmed = {}
        self._timeframe = None

        match self._subtype:
            case Subtype.OHLC:
                # - making ticks out of OHLC
                self._transformer = RestoreTicksFromOHLC()
                self._timeframe = self._subparams.get("timeframe")
                self._data_type = "ohlc"
                _id = self._data_type + str(self._timeframe)
            case Subtype.TRADE:
                self._transformer = AsTimestampedRecords()
                self._data_type = "agg_trades"
                _id = self._data_type
            case Subtype.QUOTE:
                self._transformer = AsTimestampedRecords()
                self._data_type = "orderbook"
                _id = self._data_type
            case _:
                raise ValueError(f"Unsupported subscription type: {self._subtype}")

        for i in instruments:
            self.attach_instrument(i)

        self._id = hash(_id)
        self._chunksize = chunksize  # TODO:

    def __hash__(self) -> int:
        return self._id

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, SimulationDataLoader):
            return False
        return self._data_type == other._data_type and self._timeframe == other._timeframe

    def instrument(self, idx: int) -> Instrument:
        return self._instruments.get_value_by_index(idx)

    def attach_instrument(self, instrument: Instrument) -> int:
        if not self._instruments.contains(instrument):
            self._warmed |= {f"{instrument.exchange}:{instrument.symbol}": False}
            return self._instruments.add_value(instrument)
        return -1

    def remove_instrument(self, instrument: Instrument) -> int:
        _ix = -1
        if self._instruments.contains(instrument):
            _ix = self._instruments.remove_value(instrument)
            self._warmed.pop(f"{instrument.exchange}:{instrument.symbol}")
        return _ix

    def shutdown(self):
        for s in self._instruments.values():
            self.remove_instrument(s)

    def is_instrument_removed(self, idx: int) -> bool:
        return self._instruments.is_removed(idx)

    def load(self, start: str | pd.Timestamp, end: str | pd.Timestamp, indices: List[int] | None) -> Dict[int, Iterator]:
        # - iterate over all instruments if no indices specified
        _indices = self._instruments.indices() if not indices else indices
        _r_iters = {}
        for ix in _indices:
            if _i := self._instruments.get_value_by_index(ix):
                _s = f"{_i.exchange}:{_i.symbol}"
                if self._warmup_period and not self._warmed.get(_s):
                    start = pd.Timestamp(start) - pd.Timedelta(self._warmup_period)
                    self._warmed[_s] = True

                _args = dict(
                    data_id=_s, start=start, stop=end, transform=self._transformer, data_type=self._data_type, chunksize=self._chunksize,
                )

                if self._timeframe:
                    _args["timeframe"] = self._timeframe

                _r_iters[ix] = self._reader.read(**_args)  # type: ignore
            else:
                raise IndexError(f"No instrument found for index {ix}")

        return _r_iters

In [ ]:
class SimulationDataQueue:
    _loaders: IndexedObjects

    def __init__(self):
        self._start = None
        self._stop = None
        self._current_time = None

        self._loaders = IndexedObjects()
        self._l_iterator = defaultdict(dict)

        self._event_heap = []

    @property
    def is_running(self) -> bool: return self._current_time is not None

    # def __add__(self, loader: SimulationDataLoader) -> "SimulationDataQueue":
    #     idx = self._loaders.add_value(loader)
    #     if self.is_running:
    #         self._add_chunk_to_heap(idx)
    #     return self

    # def __sub__(self, loader: SimulationDataLoader) -> "SimulationDataQueue":
    #     # ??????????
    #     return self

    def create_iterable(self, start: str | pd.Timestamp, stop: str | pd.Timestamp) -> "SimulationDataQueue":
        self._start = start
        self._stop = stop
        self._current_time = None
        return self

    def _add_chunk_to_heap(self, loader_index: int):
        chunk = self._next_chunk(loader_index)
        self._index_to_chunk_size[loader_index] = len(chunk)
        for chunk_index, event in enumerate(chunk):
            dt = event.time  # type: ignore
            heapq.heappush(self._event_heap, (dt, loader_index, chunk_index, event))

    def _next_chunk(self, index: int) -> list[Any]:
        # if index not in self._index_to_iterator:
            # self._index_to_iterator[index] = self._index_to_loader[index].load(pd.Timestamp(self._current_time, unit="ns"), self._stop)  # type: ignore
        # iterator = self._index_to_iterator[index]
        try:
            return next(iterator)
        except StopIteration:
            return []

In [546]:
sde = SimulationDataLoader(ld, "ohlc(1d)", [s1, s2], warmup_period="2d")

In [547]:
r_iters = sde.load("2023-07-01", "2023-07-02", [])
for _i, r in r_iters.items(): print(_i, list(r))

1 [[[2023-06-29T00:00:00.001000000]	30066.30000 (1000000000.0) | 30066.30000 (1000000000.0), [2023-06-29T00:24:00.000000000]	30036.10000 (1000000000.0) | 30036.10000 (1000000000.0), [2023-06-29T00:36:00.000000000]	30166.40000 (1000000000.0) | 30166.40000 (1000000000.0), [2023-06-29T00:59:59.999000000]	30070.30000 (1000000000.0) | 30070.30000 (1000000000.0), [2023-06-29T01:00:00.001000000]	30070.20000 (1000000000.0) | 30070.20000 (1000000000.0), [2023-06-29T01:24:00.000000000]	30047.80000 (1000000000.0) | 30047.80000 (1000000000.0), [2023-06-29T01:36:00.000000000]	30234.80000 (1000000000.0) | 30234.80000 (1000000000.0), [2023-06-29T01:59:59.999000000]	30154.50000 (1000000000.0) | 30154.50000 (1000000000.0), [2023-06-29T02:00:00.001000000]	30154.50000 (1000000000.0) | 30154.50000 (1000000000.0), [2023-06-29T02:24:00.000000000]	30200.00000 (1000000000.0) | 30200.00000 (1000000000.0), [2023-06-29T02:36:00.000000000]	30085.00000 (1000000000.0) | 30085.00000 (1000000000.0), [2023-06-29T02:59

In [548]:
sde.instrument(2)

BINANCE.UM:CRYPTO:ETHUSDT

In [556]:
print(sde.attach_instrument(s3))
r_iters = sde.load("2023-07-01", "2023-07-02", [])
for _i, r in r_iters.items(): print(_i, list(r))

4
2 [[[2023-07-01T00:00:00.001000000]	1933.42000 (1000000000.0) | 1933.42000 (1000000000.0), [2023-07-01T00:24:00.000000000]	1929.41000 (1000000000.0) | 1929.41000 (1000000000.0), [2023-07-01T00:36:00.000000000]	1944.69000 (1000000000.0) | 1944.69000 (1000000000.0), [2023-07-01T00:59:59.999000000]	1933.73000 (1000000000.0) | 1933.73000 (1000000000.0), [2023-07-01T01:00:00.001000000]	1933.74000 (1000000000.0) | 1933.74000 (1000000000.0), [2023-07-01T01:24:00.000000000]	1935.78000 (1000000000.0) | 1935.78000 (1000000000.0), [2023-07-01T01:36:00.000000000]	1923.33000 (1000000000.0) | 1923.33000 (1000000000.0), [2023-07-01T01:59:59.999000000]	1928.49000 (1000000000.0) | 1928.49000 (1000000000.0), [2023-07-01T02:00:00.001000000]	1928.48000 (1000000000.0) | 1928.48000 (1000000000.0), [2023-07-01T02:24:00.000000000]	1929.70000 (1000000000.0) | 1929.70000 (1000000000.0), [2023-07-01T02:36:00.000000000]	1920.22000 (1000000000.0) | 1920.22000 (1000000000.0), [2023-07-01T02:59:59.999000000]	1920.

In [559]:
print(sde.remove_instrument(s3))
print(sde.remove_instrument(s1))
r_iters = sde.load("2023-07-01", "2023-07-02", [])
for _i, r in r_iters.items(): print(_i, list(r))

-1
-1
2 [[[2023-07-01T00:00:00.001000000]	1933.42000 (1000000000.0) | 1933.42000 (1000000000.0), [2023-07-01T00:24:00.000000000]	1929.41000 (1000000000.0) | 1929.41000 (1000000000.0), [2023-07-01T00:36:00.000000000]	1944.69000 (1000000000.0) | 1944.69000 (1000000000.0), [2023-07-01T00:59:59.999000000]	1933.73000 (1000000000.0) | 1933.73000 (1000000000.0), [2023-07-01T01:00:00.001000000]	1933.74000 (1000000000.0) | 1933.74000 (1000000000.0), [2023-07-01T01:24:00.000000000]	1935.78000 (1000000000.0) | 1935.78000 (1000000000.0), [2023-07-01T01:36:00.000000000]	1923.33000 (1000000000.0) | 1923.33000 (1000000000.0), [2023-07-01T01:59:59.999000000]	1928.49000 (1000000000.0) | 1928.49000 (1000000000.0), [2023-07-01T02:00:00.001000000]	1928.48000 (1000000000.0) | 1928.48000 (1000000000.0), [2023-07-01T02:24:00.000000000]	1929.70000 (1000000000.0) | 1929.70000 (1000000000.0), [2023-07-01T02:36:00.000000000]	1920.22000 (1000000000.0) | 1920.22000 (1000000000.0), [2023-07-01T02:59:59.999000000]	1

In [560]:
print(sde._instruments)

[2]: BINANCE.UM:CRYPTO:ETHUSDT
removed: -1,-3,-4
